In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import gzip
import copy
import pickle as pk
import heapq as hp

import networkx as nx
from networkx.algorithms import bipartite, components, shortest_paths

# Importing the datasets, cleaning, transforming

## Import

**Instacart datasets** (provided) : https://www.instacart.com/datasets/grocery-shopping-2017 . The instacart datasets consist in a group of 5 different datasets, each adopting a different point of view 
* "products", a dataset based on the product point of view, describing to which aisle and department it belongs to
* "aisles", a dataset that enriches "products" by describing each aisle formally
* "departments", a dataset that enriches "products" by describing each department formally
* "orders", a dataset based on the order point of view, describing when it was purchased
* "order_products_prior", a dataset based on both the order and product point of view, associating each order with the products that were purchased

After loading the datasets, we verify whether there are any missing values. 

In [2]:
products = pd.read_csv('data/products.csv', sep=',')
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [3]:
aisles = pd.read_csv('data/aisles.csv', sep=',')
aisles.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [4]:
departments = pd.read_csv('data/departments.csv', sep=',')
departments.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [5]:
orders = pd.read_csv('data/orders.csv', sep=',')
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [6]:
history = pd.read_csv('data/order_products__prior.csv', sep=',')
history.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120.0,1.0,1.0
1,2,28985.0,2.0,1.0
2,2,9327.0,3.0,0.0
3,2,45918.0,4.0,1.0
4,2,30035.0,5.0,0.0


## Gathering all datasets

**Create a dataset with all useful information gathered : products_litteral**

We first add the names of the aisle and department in the _products_ dataset, instead of the ID, by merging the datasets :

In [7]:
products_litteral = pd.merge(pd.merge(products, aisles, on='aisle_id'), departments, on='department_id').drop(['aisle_id', 'department_id'], axis=1)
products_litteral.head()

,product_id,product_name,aisle,department
0,1,Chocolate Sandwich Cookies,cookies cakes,snacks
1,78,Nutter Butter Cookie Bites Go-Pak,cookies cakes,snacks
2,102,Danish Butter Cookies,cookies cakes,snacks
3,172,Gluten Free All Natural Chocolate Chip Cookies,cookies cakes,snacks
4,285,Mini Nilla Wafers Munch Pack,cookies cakes,snacks


Save and/or load the dataframe

In [8]:
products_litteral.to_pickle('products_litteral.pk')

In [9]:
products_litteral = pd.read_pickle('products_litteral.pk')

# Make graphs

Make a dataframe containing the orders with the details of the products

In [10]:
orders_products = pd.merge(orders, history, on='order_id').drop(['add_to_cart_order', 'reordered'], axis=1)

In [11]:
orders_products.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id
0,473747,1,prior,3,3,12,21.0,196.0
1,473747,1,prior,3,3,12,21.0,12427.0
2,473747,1,prior,3,3,12,21.0,10258.0
3,473747,1,prior,3,3,12,21.0,25133.0
4,473747,1,prior,3,3,12,21.0,30450.0


Save and/or load the dataframe

In [12]:
orders_products.to_pickle('orders_products.pk')

In [13]:
orders_products = pd.read_pickle('orders_products.pk')

In [14]:
orders_products.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id
0,473747,1,prior,3,3,12,21.0,196.0
1,473747,1,prior,3,3,12,21.0,12427.0
2,473747,1,prior,3,3,12,21.0,10258.0
3,473747,1,prior,3,3,12,21.0,25133.0
4,473747,1,prior,3,3,12,21.0,30450.0


Now, we need to select only food-related products.

Because dataframes containing all the products and orders are really big to handle, we separate products by aisle and store each aisle's dataframe separately.

In [15]:
food_depts = ['bulk', 'meat seafood', 'snacks', 'beverages', 'frozen', 'dairy eggs', 'canned goods', 'dry goods pasta', 'produce', 'bakery', 'deli', 'breakfast', 'alcohol', 'pantry', 'international']

In [16]:
food_aisles = {row.aisle for i, row in products_litteral.iterrows() if row.department in food_depts}

Save and/or load the dataframe

In [17]:
with open('food_aisles.pk', 'wb') as file:
    pk.dump(food_aisles, file)

In [18]:
with open('food_aisles.pk', 'rb') as file:
    food_aisles = pk.load(file)

For each aisle, make the dataframe containing all its products' information (and save/load)

In [19]:
by_aisle = {}
for aisle in food_aisles:
    by_aisle[aisle] = products_litteral[products_litteral.aisle==aisle]
    by_aisle[aisle].to_pickle('by_aisle_'+aisle+'.pk')

In [20]:
by_aisle = {}
for aisle in food_aisles:
    by_aisle[aisle] = pd.read_pickle('by_aisle_'+aisle+'.pk')

In [21]:
# this is needed otherwise the kernel will end up dying because it is overloaded
del(products_litteral)

For each aisle, make the dataframe containing all orders (and save/load).

In [22]:
orders_products_by_aisle = {}
for aisle in food_aisles:
    orders_products_by_aisle[aisle] = pd.merge(orders_products, by_aisle[aisle], on='product_id')[['user_id', 'product_id', 'order_id', 'product_name']]


KeyboardInterrupt: 

Now count the number of orders of each product for each user (and save/load).

In [ ]:
orders_users_by_aisle = {}
for aisle in food_aisles:
    orders_users_by_aisle[aisle] = orders_products_by_aisle[aisle].groupby(['user_id', 'product_id', 'product_name']).count().rename(columns={'order_id':'nb_orders'})
    orders_users_by_aisle[aisle].to_pickle('orders_users_by_aisle_'+aisle+'.pk')
    print(orders_users_by_aisle[aisle].head())

In [ ]:
orders_users_by_aisle = {}
for aisle in food_aisles:
    orders_users_by_aisle[aisle] = pd.read_pickle('orders_users_by_aisle_'+aisle+'.pk')
    print(orders_users_by_aisle[aisle].head())

We want to normalize the number of orders (_i.e._ divide by the total number of food buys for each user) because we are omre interested in the preference of users for some products (relatively to other products) than actual number of orders.

To do this, we first sum the number of buys of each user of each aisle, and then sum over aisles for each user.

In [ ]:
tot_user_by_aisle = {}
for aisle in food_aisles:
    orders_users_by_aisle[aisle] = orders_users_by_aisle[aisle].reset_index()
    tot_user_by_aisle[aisle] = orders_users_by_aisle[aisle][['user_id', 'nb_orders']].groupby('user_id').sum().rename(columns={'nb_orders':'tot_orders'})
    print(tot_user_by_aisle[aisle].head())

This dataframe contains the total number of buys for each user in all food aisles:

In [ ]:
tot_user = pd.concat(tot_user_by_aisle[aisle] for aisle in food_aisles).groupby('user_id').sum()
tot_user.head()

Save and/or load the dataframe

In [ ]:
tot_user.to_pickle('tot_user.pk')

In [ ]:
tot_user = pd.read_pickle('tot_user.pk')

Use this total number of buys to normalise counts for each aisle:

In [ ]:
tot_user.replace(0, np.nan)
tot_user.columns = ['nb_orders']
orders_users_norm_by_aisle = {}
for aisle in food_aisles:
    orders_users_by_aisle[aisle] = orders_users_by_aisle[aisle].set_index(['user_id', 'product_id'])
    orders_users_norm_by_aisle[aisle] = orders_users_by_aisle[aisle][['nb_orders']] / tot_user
    orders_users_norm_by_aisle[aisle].to_pickle('orders_users_norm_by_aisle_'+aisle+'.pk')

In [ ]:
orders_users_norm_by_aisle = {}
for aisle in food_aisles:
    orders_users_norm_by_aisle[aisle] = pd.read_pickle('orders_users_norm_by_aisle_'+aisle+'.pk')

In [ ]:
del(orders_products_by_aisle)

Now is time to build the graphs.

First, we create (for each aisle) a bipartite graph where all products are connected to all users who have ordered them at least once. The weight of the edge between one user and a product they have bought is the normalized number of times they have bought this product.

In [ ]:
graphs = {aisle: nx.Graph() for aisle in food_aisles}

# first, create the nodes:
for aisle in food_aisles:
    # products nodes
    for prod in by_aisle[aisle]['product_id'].unique():
        graphs[aisle].add_node(prod, bipartite=0)
    # user nodes (use negative id to distinguish them from product ids)
    for user in orders_users_norm_by_aisle[aisle].reset_index().user_id.unique():
        graphs[aisle].add_node(-user, bipartite=1)


In [ ]:
# second, add edges between the nodes
for aisle in food_aisles:
    for (user, prod), row in orders_users_norm_by_aisle[aisle].iterrows():
        graphs[aisle].add_edge(prod, -user, weight=row['nb_orders'])
    nx.write_gpickle(graphs[aisle], "users_and_prods_graph_"+aisle+'.gpk')
    print('Done', aisle)

In [ ]:
graphs = {}
for aisle in food_aisles:
    graphs[aisle] = nx.read_gpickle("users_and_prods_graph_"+aisle+'.gpk')
    print('Loaded', aisle)

dept = 'produce'
graphs[dept] = nx.Graph()
for prod in by_dept[dept]['product_id'].unique():
    graphs[dept].add_node(prod, bipartite=0)
for user in orders.user_id.unique():
    graphs[dept].add_node(-user, bipartite=1)
for (user, prod, name), row in orders_users_by_dept[dept].iterrows():
    graphs[dept].add_edge(prod, -user, weight=row['nb_orders'])

In [ ]:
del(graphs)

Now we project the bipartite graph on the _products only_.

The weight between two products is the average (normalized) number of orders of the two products over all common users, multiplied by the square of the number of users in common.

In [ ]:
# define the weight of inter-product edges
def my_weight(G, u, v):
    w = 0
    n = 0
    for nbr in set(G[u]) & set(G[v]):
        n += 1
        w += (G[u][nbr].get('weight', 1) + G[v][nbr].get('weight', 1)) / 2
    return w * n   # divide by n for average and multiply by n**2

In [ ]:
# project on products using this weight function
prod_graphs = {}
for aisle in food_aisles:
    #print('\n', dept)
    G = nx.read_gpickle("users_and_prods_graph_"+aisle+'.gpk')
    prod_graphs[aisle] = bipartite.generic_weighted_projected_graph(G, {n for n, d in G.nodes(data=True) if d['bipartite']==0}, weight_function=my_weight)
    nx.write_gpickle(prod_graphs[aisle], 'prod_graphs_'+aisle+'.gpk')
    print('Done', aisle)

In [ ]:
del(prod_graphs)

In [ ]:
# just have a look
c=0
for aisle in food_aisles:
    c+=1
    if c>3:   # don't print all graphs, just a few
        break
    print('\n', aisle)
    G = nx.read_gpickle('prod_graphs_'+aisle+'.gpk')
    print(nx.info(G))
    plt.figure()
    nx.draw(G, with_labels=True,  alpha = 0.8)
    plt.title(aisle)
    plt.figure()
    nx.draw_circular(G, with_labels=True,  alpha = 0.8)
    plt.title(aisle)
    plt.figure()
    nx.draw_spectral(G, with_labels=True,  alpha = 0.8)
    plt.title(aisle)


We notice that these graphs are very dense. Therefore, it might be useful to remove small weight edges (if only very few users have bought a pair of products, they do not deserve to be linked).

Let's have a look at the edges weights:

In [ ]:
def look_at_edges(G):
    edge_weights = [d['weight'] for (u, v, d) in G.edges(data=True)] # if d['weight'] < 200
    plt.figure()
    plt.hist(edge_weights, bins=100, log=True)
    print(np.quantile(edge_weights, [0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 0.7, 0.9, 0.99]))

In [ ]:
c=0
for aisle in food_aisles:
    c+=1
    if c>7:
        break
    look_at_edges(nx.read_gpickle('prod_graphs_'+aisle+'.gpk'))

The distribution of edges weights is extremely high on for very low weights. We choose to use the 30th percentile (_i.e._ remove the 30% lowest edges).

Define the function that, from a graph of products, removes the exceedingly numerous small edges (but reduces the percentile if too many products get disconnected from the graph). It also uses the inverse of the original weights as weights in order to enable looking for a shortest path.

In [ ]:
def make_final_graph(G, aisle='None', threshold_quantile=0.3, verbose=False,save=False):
    print(aisle)
    edge_weights = [d['weight'] for (u, v, d) in G.edges(data=True)]
    lost = len(G)
    threshold_quantile = 2 * threshold_quantile
    stop = False
    threshold = 0
    while lost > 0.15 * len(G) and not stop:            # while too many nodes get disconnected, ...
        threshold_quantile = threshold_quantile / 2     # ... reduce percentile of edges removed
        new_threshold = np.quantile(edge_weights, threshold_quantile)
        if threshold == new_threshold:   # if reducing the percentile ceases to change the threshold, stop
            stop=True
        else:
            threshold = new_threshold
        thresholded_inv_edges = []
        C=0
        for (u, v, d) in G.edges(data=True):
            C+=1
            if d['weight'] > threshold: # select heavy enough eg-dges
                nd = d.copy()
                nd['weight'] = 1. / d['weight'] # use inverse of weight
                thresholded_inv_edges.append((u,v,nd))
        # make graph from selected edges:
        thresholded_inv = nx.Graph(thresholded_inv_edges)
        lost = len(G)-len(thresholded_inv)   # number of nodes that got disconnected
    print('Lost products: {} of {}'.format(len(G)-len(thresholded_inv), len(G)))
    print("Used quantile: {}".format(threshold_quantile))
    if verbose:
        print('Avg shortest Path:', shortest_paths.generic.average_shortest_path_length(thresholded_inv))
        print('Avg weighted shortest Path:', shortest_paths.generic.average_shortest_path_length(thresholded_inv, weight='weight'))
        print(nx.info(G))
        print(nx.info(thresholded_inv))
    if save:
        nx.write_gpickle(thresholded_inv, 'thresholded_inv_'+aisle+'.gpk')
    return thresholded_inv


Make the graphs to be used for product recommendation.

In [ ]:
for aisle in food_aisles:
    make_final_graph(nx.read_gpickle('prod_graphs_'+aisle+'.gpk'), aisle, save=True)
    print('Done', aisle)
    print()

Make the function that, given a product (supposed not too healthy), outputs a recommended product (from the same aisle). The recommended product is selected among the 10 closest neighbours of the original product, based on a score taking into account distance to original, healthiness, and fidelity (~satisfaction) of consumers.

In [ ]:
def find_recommendation_path(source_id, G=None, aisle=None, health_thresh=2, product_scores=None, alpha=.3, beta=.2):
    """alpha is coefficient for healthiness, beta coefficient for fidelity, and 1-alpha-beta is coefficient for distance."""
    if G is None:
        G = nx.read_gpickle('thresholded_inv_'+aisle+'.gpk')
    if product_scores is None:
        product_scores = pd.read_pickle('GOOD_product_scores.pk')
    
    # select target products defined as having a healthiness greater than health_thresh
    targets = set()
    for x in G.nodes:
        if x in product_scores.index and product_scores.at[x, "healthiness"] >= health_thresh:
            targets.add(x)
    print('Nb of targets:', len(targets))
    
    # do not allow paths through products that are even less healthy than source:
    G_ok = G.subgraph({x for x in G.nodes if x in product_scores.index and product_scores.at[x, "healthiness"] >= product_scores.at[source_id, "healthiness"]})
    
    # find all shortest paths from source:
    pred, lengths = nx.dijkstra_predecessor_and_distance(G_ok, source_id, weight='weight')
    
    # find 10 closest target neighbours:
    candidates = [(-np.inf, None)]
    sep = []
    for candidate in targets:
        if candidate not in lengths:   # ie it is not connected to source
            sep.append(candidate)
        if candidate in lengths and - lengths[candidate] > candidates[0][0]:   # ie candidate is closer than all current candidates
            if len(candidates) >= 10:
                hp.heappop(candidates)
            hp.heappush(candidates, (- lengths[candidate], candidate))
    if len(sep) == len(targets):
        print("not connected to any healthy product")
        return
    if candidates == [(-np.inf, None)]:
        print('No path to healthy products')
        return
    
    # find the distances, healthinesses and fidelities of all ten candidates for min-max scaling
    fids = []
    dists = []
    healths = []
    for (mdist, candidate) in candidates:
        if mdist == -np.inf:
            continue
        fids.append(product_scores.at[candidate, 'fidelity'])
        dists.append(- mdist)
        healths.append(product_scores.at[candidate, 'healthiness'])
    max_fid = max(fids)
    min_fid = min(fids)
    max_dist = max(dists)
    min_dist = min(dists)
    max_health = max(healths)
    min_health = min(healths)
    
    rec = (0, None)
    for (mdist, candidate) in candidates:
        if mdist == -np.inf:
            continue
        # perform min-max scaling
        fid = ((product_scores.at[candidate, 'fidelity'] - min_fid) / (max_fid - min_fid)) if max_fid != min_fid else 0
        dist = ((max_dist + mdist) / (max_dist - min_dist)) if max_dist != min_dist else 0
        health = ((product_scores.at[candidate, "healthiness"] - min_health) / (max_health - min_health)) if max_health != min_health else 0
        # compute aggregate score
        score = (1+health)**alpha * (1+fid)**beta * (1+dist)**(1-alpha-beta)
        if score > rec[0]:   # choose best scoring product
            rec = (score, candidate)
    
    # now build path between recommended product and source
    orig = rec[1]
    path = [orig]
    while orig != source_id:
        orig = pred[orig][0]
        path.append(orig)
    
    path.reverse()
    return path

In [ ]:
find_recommendation_path(3265, aisle='packaged produce')

### Recommend products

Let's find some real recommendations for some products that users from the cluster 0 (considered the least healthy) have made:

Get orders from "unhealthy" users:

In [ ]:
orders_products = pd.read_pickle('orders_products.pk')
with open('THE_GOOD_users_cluster.pk', 'rb') as f:
    users_cluster = pk.load(f)
unhealthy = set(u+1 for u in range(len(users_cluster)) if users_cluster[u]==0)
unhealthy_orders = orders_products[orders_products.user_id.isin(unhealthy)]
unhealthy_food_orders = unhealthy_orders[unhealthy_orders.apply(lambda row: prod_lit.at[row.product_id, 'aisle'] in food_aisles, axis=1)]

Sample some orders:

In [ ]:
some_unhealthy_orders = unhealthy_orders.sample(100)[['order_id', 'user_id', 'product_id']]

In [ ]:
some_unhealthy_orders.head()

In [ ]:
prod_lit = products_litteral.set_index('product_id')
prod_lit.head()

Find recommendations for selected orders:

In [ ]:
for _, row in some_unhealthy_orders.iterrows():
    try:
        print('For user ', row.user_id)
        print('Path from', prod_lit.at[row.product_id, 'product_name'], 'with healthiness', product_scores.at[row.product_id, 'healthiness'])
        path = find_recommendation_path(row.product_id, aisle=prod_lit.at[row.product_id, 'aisle'], product_scores=product_scores)
        target = path[-1]
        print('To', prod_lit.at[target, 'product_name'], 'with healthiness', product_scores.at[target, 'healthiness'])
        print()
    except:
        print('============================= Echec: For user ', row.user_id)
        print('Path from', prod_lit.at[row.product_id, 'product_name'], 'with healthiness', product_scores.at[row.product_id, 'healthiness'])
        print()
        

### See whether the healthiness measure makes sense

Compute average healthiness per aisle:

In [ ]:
healthiness_per_aisle = {}
for aisle in food_aisles:
    healthiness_per_aisle[aisle] = pd.merge(by_aisle[aisle], product_scores, left_on='product_id', right_on='product_id')

In [ ]:
for aisle in food_aisles:
    print(aisle, healthiness_per_aisle[aisle].mean()['healthiness'])
    print("====================")

See what proportion of products of each aisle are considered "target" products:

In [ ]:
for aisle in food_aisles:
    t = 0
    g = 0
    lst = []
    for ind in by_aisle[aisle]['product_id']:
        if ind in product_scores.index:
            lst.append(product_scores.at[ind, 'healthiness'])
        if ind in product_scores.index and product_scores.at[ind, 'healthiness']>=2:
            g += 1
        t+=1
    print(aisle, float(g)/t, g, t)

This is necessary to use dataframe from Pierre's notebook:

In [ ]:
product_scores = pd.read_pickle('GOOD_product_scores')

In [ ]:
product_scores.columns=['healthiness', 'count', 'fidelity']

In [ ]:
product_scores.set_index(product_scores.index.astype(int)).to_pickle('GOOD_product_scores.pk')

In [ ]:
product_scores = pd.read_pickle('GOOD_product_scores.pk')